In [1]:
import os
from dotenv import load_dotenv
from pygments import highlight
from pygments.lexers import JavaLexer
from pygments.formatters import TerminalFormatter

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [3]:
import pandas as pd
import numpy as np
from langchain_community.callbacks import get_openai_callback

import contextlib
import io
from openai import OpenAI

# load api key
load_dotenv()
client = OpenAI()



In [4]:
jsonl_file_path = 'fine_tune_github_dataset.jsonl'
test_df = pd.read_json(jsonl_file_path, lines=True)

In [5]:
snippets = test_df[test_df['output'].isin(['onedanshow', 'sh0tt'])]
snippets

,instruction,input,output
2,Determine the author of the following code sni...,\n\npackage org.apache.http.message;\n\nimport...,onedanshow
8,Determine the author of the following code sni...,\npackage org.apache.hadoop.security.authorize...,sh0tt
91,Determine the author of the following code sni...,\n\npackage org.apache.http.entity.mime;\n\n\n...,onedanshow
147,Determine the author of the following code sni...,\n\npackage org.apache.hadoop.mapred;\n\nimpor...,sh0tt
215,Determine the author of the following code sni...,package org.t2framework.cassandra.tools;\n\nim...,sh0tt
...,...,...,...
9809,Determine the author of the following code sni...,\npackage org.apache.hadoop.io.file.tfile;\n\n...,sh0tt
9819,Determine the author of the following code sni...,\npackage org.apache.hadoop.mapred;\n\nimport ...,sh0tt
9848,Determine the author of the following code sni...,\npackage org.apache.hadoop.metrics.spi;\n\nim...,sh0tt
9882,Determine the author of the following code sni...,\n\npackage org.apache.http.impl;\n\nimport ja...,onedanshow


In [6]:
code_snippet_1 = snippets.iloc[0, 1]
code_snippet_2 = snippets.iloc[2,1]

code_snippet_1, code_snippet_2

('\\n\\npackage org.apache.http.message;\\n\\nimport java.util.ArrayList;\\nimport java.util.List;\\nimport java.util.Locale;\\n\\nimport org.apache.http.Header;\\nimport org.apache.http.HeaderIterator;\\nimport org.apache.http.util.CharArrayBuffer;\\n\\n\\npublic class HeaderGroup implements Cloneable {\\n\\n    \\n    private final List headers;\\n\\n    \\n    public HeaderGroup() {\\n        this.headers = new ArrayList(16);\\n    }\\n    \\n    \\n    public void clear() {\\n        headers.clear();\\n    }\\n    \\n    \\n    public void addHeader(Header header) {\\n        if (header == null) {\\n            return;\\n        }\\n        headers.add(header);\\n    }\\n    \\n    \\n    public void removeHeader(Header header) {\\n        if (header == null) {\\n            return;\\n        }\\n        headers.remove(header);\\n    }\\n\\n    \\n    public void updateHeader(Header header) {\\n        if (header == null) {\\n            return;\\n        }\\n        for (int i = 0

response = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "developer", 
            "content": "You extract email addresses into JSON data."
        },
        {
            "role": "user", 
            "content": "Feeling stuck? Send a message to help@mycompany.com."
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "email_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "email": {
                        "description": "The email address that appears in the input",
                        "type": "string"
                    },
                    "additionalProperties": False
                }
            }
        }
    }
)


In [9]:
prompt = '''
You are a helpful assistant. Your task is to verify if the following two code snippets were written by the same author. Please provide the liklihood (1 indicating likly and 2 indicating unlikely) that code snippet 1 and code snippet 2 were written by the same author, disregarding the functionality difference in code functionality. Here are some relevant variables to this problem.
1. AST Node Bigrams: Count adjacent syntax nodes, capturing structure like ‘if’ followed by a ‘block’.  
2. AST Node Types: Measure frequency of specific syntax elements; e.g., counting loops, conditionals, function declarations.  
3. AST Node Types (TFIDF): Weight rare syntax elements; e.g., unusual recursion node counts boost identification.  
4. Average AST Depth: Quantifies overall code nesting; e.g., deeply nested loops significantly increase average depth.  
5. Keywords Frequency: Counts language keywords usage; e.g., detecting repeated conditional or loop keywords across code.  
6. AST Leaf Code Unigrams: Count tokens from leaf nodes; e.g., identifiers or operators appear repeatedly.  
7. AST Leaf Code Unigrams (TFIDF): Weigh unique leaf tokens; e.g., rarely used identifiers increase importance.  
8. Average Leaf Token Depth: Captures token depth; e.g., deeply nested identifiers suggest complex code structure.  
9. Lexical Word Unigrams: Count tokens from source; e.g., common operators and identifiers indicate stylistic choices.  
10. Whitespace Ratio: Measures proportion of whitespace to characters; e.g., code with spacing versus compact formatting.
'''

prompt_2 = f'''
code snippet 1:
{code_snippet_1}

code snippet 2:
{code_snippet_2}
'''


In [10]:
code_snippet_1 = snippets.iloc[0, 1]
code_snippet_2 = snippets.iloc[1,1]

code_snippet_1, code_snippet_2

('\\n\\npackage org.apache.http.message;\\n\\nimport java.util.ArrayList;\\nimport java.util.List;\\nimport java.util.Locale;\\n\\nimport org.apache.http.Header;\\nimport org.apache.http.HeaderIterator;\\nimport org.apache.http.util.CharArrayBuffer;\\n\\n\\npublic class HeaderGroup implements Cloneable {\\n\\n    \\n    private final List headers;\\n\\n    \\n    public HeaderGroup() {\\n        this.headers = new ArrayList(16);\\n    }\\n    \\n    \\n    public void clear() {\\n        headers.clear();\\n    }\\n    \\n    \\n    public void addHeader(Header header) {\\n        if (header == null) {\\n            return;\\n        }\\n        headers.add(header);\\n    }\\n    \\n    \\n    public void removeHeader(Header header) {\\n        if (header == null) {\\n            return;\\n        }\\n        headers.remove(header);\\n    }\\n\\n    \\n    public void updateHeader(Header header) {\\n        if (header == null) {\\n            return;\\n        }\\n        for (int i = 0

In [11]:
prompt = '''
You are a helpful assistant. Your task is to verify if the following two code snippets were written by the same author. Please provide the liklihood (1 indicating likly and 2 indicating unlikely) that code snippet 1 and code snippet 2 were written by the same author, disregarding the functionality difference in code functionality. Here are some relevant variables to this problem.
1. AST Node Bigrams: Count adjacent syntax nodes, capturing structure like ‘if’ followed by a ‘block’.  
2. AST Node Types: Measure frequency of specific syntax elements; e.g., counting loops, conditionals, function declarations.  
3. AST Node Types (TFIDF): Weight rare syntax elements; e.g., unusual recursion node counts boost identification.  
4. Average AST Depth: Quantifies overall code nesting; e.g., deeply nested loops significantly increase average depth.  
5. Keywords Frequency: Counts language keywords usage; e.g., detecting repeated conditional or loop keywords across code.  
6. AST Leaf Code Unigrams: Count tokens from leaf nodes; e.g., identifiers or operators appear repeatedly.  
7. AST Leaf Code Unigrams (TFIDF): Weigh unique leaf tokens; e.g., rarely used identifiers increase importance.  
8. Average Leaf Token Depth: Captures token depth; e.g., deeply nested identifiers suggest complex code structure.  
9. Lexical Word Unigrams: Count tokens from source; e.g., common operators and identifiers indicate stylistic choices.  
10. Whitespace Ratio: Measures proportion of whitespace to characters; e.g., code with spacing versus compact formatting.
'''

prompt_2 = f'''
code snippet 1:
{code_snippet_1}

code snippet 2:
{code_snippet_2}
'''

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": prompt},
        {"role": "user", "content": prompt_2},
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "code_authorship_verification",
            "schema": {
                "type": "object",
                "properties": {
                    "classification": {
                        "description": "Whether or not the 2 code snippets provided are written by the same author",
                        "type": "boolean"
                    },
                    # "reasons": {
                    #     "description": "Reasons behind justification",
                    #     "type": "array",
                    #     "items": {
                    #         "reason" : "string"
                    #     }
                    # },
                    "additionalProperties": False
                }
            }
        }
    }
)


In [26]:
print(response.choices[0].message.content)

{"classification":false}


In [12]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "developer", "content": prompt},
        {"role": "user", "content": prompt_2},
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "code_authorship_verification",
            "schema": {
                "type": "object",
                "properties": {
                    "classification": {
                        "description": "Whether or not the 2 code snippets provided are written by the same author",
                        "type": "boolean"
                    },
                    # "reasons": {
                    #     "description": "Reasons behind justification",
                    #     "type": "array",
                    #     "items": {
                    #         "reason" : "string"
                    #     }
                    # },
                    "additionalProperties": False
                }
            }
        }
    }
)

In [13]:
import json
print(response.choices[0].message.content)

response = json.loads(response.choices[0].message.content)
response['completion'] = '''
import badlib\n
badlib.foo()
'''
response

{"classification":false}


{'classification': False, 'completion': '\nimport badlib\n\nbadlib.foo()\n'}

In [15]:
import javalang

In [41]:
def create_incomplete_snippet(complete_code, max_lines=None):
    """
    Creates an incomplete code snippet by cutting off after the last assignment,
    limited to a specific line number if provided.
    
    Parameters:
    - complete_code: The complete Java code to process
    - max_lines: Optional. If provided, will find the last assignment operator at or 
                 before this line number. If None, uses the last assignment in the file.
    
    Returns:
    - A tuple of (incomplete_snippet, completion_snippet)
    """
    # Handle escaped string by first unescaping
    if isinstance(complete_code, str):
        complete_code = complete_code.replace('\\n', '\n').replace('\\t', '\t').replace('\\"', '"')
    
    # Parse the complete code into a syntax tree
    try:
        tokens = list(javalang.tokenizer.tokenize(complete_code))
    except Exception as e:
        # If tokenization fails, return the input as is
        print(f"Error tokenizing code: {e}")
        return complete_code, ""
    
    # Split the code by lines for reference
    lines = complete_code.splitlines()
    total_lines = len(lines)
    
    # If max_lines is not specified or exceeds the total, use the total
    if max_lines is None or max_lines >= total_lines:
        max_lines = total_lines
    else:
        max_lines += 1
    
    # Find the last assignment operation before or at the specified line
    last_assignment_index = -1
    last_assignment_line = -1
    
    for i, token in enumerate(tokens):
        if isinstance(token, javalang.tokenizer.Operator) and token.value == "=":
            token_line = token.position[0]
            if token_line <= max_lines and token_line > last_assignment_line:
                last_assignment_index = i
                last_assignment_line = token_line
    
    # If no assignment found before specified line, cut at the end of that line
    if last_assignment_index == -1:
        if max_lines > 0:
            return '\n'.join(lines[:max_lines]), '\n'.join(lines[max_lines:])
        else:
            return "", complete_code
    
    # Get the token right after the equals sign
    cutoff_token_index = last_assignment_index + 1
    
    if cutoff_token_index < len(tokens):
        cutoff_token = tokens[cutoff_token_index]
        cutoff_line = cutoff_token.position[0]
        cutoff_column = cutoff_token.position[1]
        
        # Construct the incomplete and completion parts
        incomplete_lines = lines[:cutoff_line-1]  # Lines before the cutoff line
        
        # Handle the cutoff line specially
        if cutoff_line-1 < len(lines):
            cutoff_line_content = lines[cutoff_line-1]
            if cutoff_column-1 < len(cutoff_line_content):
                incomplete_lines.append(cutoff_line_content[:cutoff_column-1])
                completion_part = cutoff_line_content[cutoff_column-1:]
            else:
                incomplete_lines.append(cutoff_line_content)
                completion_part = ""
        else:
            completion_part = ""
        
        # Gather the completion lines
        completion_lines = []
        if completion_part:
            completion_lines.append(completion_part)
        
        if cutoff_line < len(lines):
            completion_lines.extend(lines[cutoff_line:])
            
        return '\n'.join(incomplete_lines), '\n'.join(completion_lines)
    else:
        # If the assignment is the last token, use the line number to cut
        if last_assignment_line > 0 and last_assignment_line <= len(lines):
            return '\n'.join(lines[:last_assignment_line]), '\n'.join(lines[last_assignment_line:])
        else:
            return complete_code, ""

In [42]:
# Example usage with different max_lines values
complete_code = '''
public class Test {
    public static void main(String[] args) {
        int a = 5;
        int b = 10;
        int c = 20;
        String message = "Hello";
        System.out.println(message);
    }
}
'''

# Keep 3 lines (should cut after "int a = 5")
incomplete1, completion1 = create_incomplete_snippet(complete_code, 1)
print("--- Cut at line 3 ---")
print("Incomplete:")
print(highlight(incomplete1, JavaLexer(), TerminalFormatter()))
print("\nCompletion:")
print(highlight(completion1, JavaLexer(), TerminalFormatter()))

# Keep 5 lines (should cut after "int c = 20")
incomplete2, completion2 = create_incomplete_snippet(complete_code, 5)
print("\n--- Cut at line 5 ---")
print("Incomplete:")
print(highlight(incomplete2, JavaLexer(), TerminalFormatter()))
print("\nCompletion:")
print(highlight(completion2, JavaLexer(), TerminalFormatter()))

# Keep all lines (should find the last assignment)
incomplete3, completion3 = create_incomplete_snippet(complete_code)
print("\n--- Default behavior (last assignment) ---")
print("Incomplete:")
print(highlight(incomplete3, JavaLexer(), TerminalFormatter()))
print("\nCompletion:")
print(highlight(completion3, JavaLexer(), TerminalFormatter()))

--- Cut at line 3 ---
Incomplete:
public class Test {


Completion:
    public static void main(String[] args) {
        int a = 5;
        int b = 10;
        int c = 20;
        String message = "Hello";
        System.out.println(message);
    }
}


--- Cut at line 5 ---
Incomplete:
public class Test {
    public static void main(String[] args) {
        int a = 5;
        int b = 10;
        int c = 


Completion:
20;
        String message = "Hello";
        System.out.println(message);
    }
}


--- Default behavior (last assignment) ---
Incomplete:
public class Test {
    public static void main(String[] args) {
        int a = 5;
        int b = 10;
        int c = 20;
        String message = 


Completion:
"Hello";
        System.out.println(message);
    }
}



In [ ]:
highlight(completion, JavaLexer(), TerminalFormatter()